In [ ]:
import torch
%load_ext autoreload
import neptune.new as neptune
from trainers import timegan_generate_data, rtsgan_generator, rgan_generator
from utils import *

# Makes sure the same testset is generated every time
np.random.seed(42)
alpha = 0.7
noise = 0
testset = DatasetSinus(num=600, seq_len=100, alpha=alpha, noise=noise)

In [ ]:
plt.plot(testset[:][0][2]);

In [ ]:
%autoreload
# Models
from architectures.timegan_cnn_DG import *
#from architectures.timegan_cnn_D import *
#from architectures.timegan_cnn_G import *
#from architectures.timegan_cnn_DGER import *
#from architectures.RGAN import *
#from architectures.RTSGAN import *

print(f"Loading architecture: {ID}")

In [ ]:
id = "TIMEGAN-178" # RTSGAN-66
#id = "RTSGAN-66"
project_name = "timeGAN" # RTSGAN
#project_name = "RTSGAN"

run = neptune.init_run(
                with_id=id, # "TIMEGAN-84"
                project="kohmann/" + project_name,
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3YjFjNGY5MS1kOWU1LTRmZjgtOTNiYS0yOGI2NDdjZGYzNWUifQ==",
                   )
params = run["parameters"].fetch()
params["device"] = "cpu"
params["testset_size"] = len(testset)
#params["model_name"] = "model_checkpoint.pt"

if "TimeGAN" in ID:
    model = TimeGAN(params)
elif ID == "RGAN":
    model = RGAN(params)
elif ID == "RTSGAN":
    model = RTSGAN(params)
else:
    raise ValueError
model = restore_weights(model, run)


In [ ]:
%autoreload
np.random.seed(42)
#fake_data = rtsgan_generator(model, params, eval=True)
fake_data = timegan_generate_data(model, torch.tensor(testset.T), params["max_seq_len"], params["Z_dim"])

#### Evaluation methods

In [ ]:
run = neptune.init_run(
    project="kohmann/Evaluation",
    name=ID,
    description="",
    #source_files=["architectures/RTSGAN.py"],
    capture_hardware_metrics=False,
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3YjFjNGY5MS1kOWU1LTRmZjgtOTNiYS0yOGI2NDdjZGYzNWUifQ==",
)
run["model_id"] = ID + "-" + id.split('-')[-1]

from metrics import compare_sin3_generation, sw_approx # sinkhorn_distance, MMD,
np.random.seed(42)
testset2 = DatasetSinus(num=1000, seq_len=100, alpha=alpha, noise=noise)
mse_error = compare_sin3_generation(fake_data, 0.7, 0)
print(f"MSE Error: {mse_error:.5f}")
x = torch.tensor(fake_data)
y = testset[:][0]
y_2 = testset2[:][0]
#wass_dist = sinkhorn_distance(x,y)
#mmd = MMD(x,y)
sw_baseline = sw_approx(y,y_2)
sw = sw_approx(y,x)

run["numeric_results/num_test_samples"] = len(testset)
run["numeric_results/sin3_generation_MSE_loss"] = mse_error
#run["numeric_results/wasserstein_distance_mean"] = wass_dist.mean()
#run["numeric_results/wasserstein_distance_std"] = wass_dist.std()
run["numeric_results/SW"] = sw.item()
run["numeric_results/SW_baseline"] = sw_baseline.item()

r = np.array([data[0].numpy() for data in testset])
#sin = None
#f_pca = visualization(r[:,:,sin][:,:,None], fake_data[:,:,sin][:,:,None], 'umap')
run["PCA"].upload(visualization(r, fake_data, 'pca'))
run["tSNE"].upload(visualization(r, fake_data, 'tsne'))
run["UMAP"].upload(visualization(r, fake_data, 'umap'))
run.stop()

In [ ]:
%autoreload
from metrics import compare_sin3_generation, sinkhorn_distance, MMD
mse_error = compare_sin3_generation(fake_data, 0.7, 0)
print(f"MSE Error: {mse_error:.5f}")
x = torch.tensor(fake_data, dtype=torch.float32)
y = testset[:][0]
wass_dist = sinkhorn_distance(x,y, blur=0.01)
mmd = MMD(x,y)
print(f"Mean Wasserstein/Sinkhorn distance: {wass_dist.mean():.8f} ± {wass_dist.std():.8f}")
print(f"Mean MMD: {mmd.mean():.6f} ± {mmd.std():.6f}")

In [ ]:
wass_dist.mean().numpy(), wass_dist.numpy().mean()

In [ ]:
x = torch.tensor(fake_data, dtype=torch.float32)[0]
y = testset[:][0][0]

In [ ]:
def sw_approx(mu: torch.Tensor, nu: torch.Tensor) -> float:

    def m_2(X):
        return torch.mean(torch.pow(X, 2), dim=0)

    m_mu = torch.mean(mu, dim=0)
    m_nu = torch.mean(nu, dim=0)
    ### First lets compute d:=W2{N(0, m2(µd_bar)), N(0, m2(νd_bar))}
    # Centered version of mu and nu
    mu_bar = mu - m_mu
    nu_bar = nu - m_nu
    # Compute Wasserstein beetween two centered gaussians
    W = torch.pow(torch.sqrt(m_2(mu_bar)) - torch.sqrt(m_2(nu_bar)), 2)

    ## Compute the mean residuals
    d = mu.size(1)
    res = (1 / d) * torch.pow(m_mu - m_nu, 2)

    ## Approximation of the Sliced Wasserstein
    return torch.norm(W + res, p=2)

In [ ]:
from utils import *
N = 10**4
testset1 = DatasetSinus(num=N, seq_len=100, alpha=0.7, noise=0)[:][0]
testset2 = DatasetSinus(num=N, seq_len=100, alpha=0.7, noise=0)[:][0]
plt.plot(testset1[0]);

In [ ]:

f = [sw_approx(testset1[:10**n].reshape(-1, 3*100), testset2[:10**n].reshape(-1, 3*100)).item() for n in range(1, 5)]
plt.plot([10**n for n in range(1, 5)], f)

In [ ]:
sw_approx(testset1[:1000], testset2[:1000]).item()

In [ ]:
[10**n for n in range(1, 5)]

In [ ]:
f

In [ ]:
f

In [ ]:
f

In [ ]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
import time

## First sample from two different distributions
m1 = torch.tensor([1., 2., 3.])
m2 = torch.tensor([4., 5., 6.])
sig1 = torch.tensor([[1., 1., 1.], [1., 2., 2.], [1., 2., 3.]])
sig2 = torch.eye(3)
mu_distrib = MultivariateNormal(m1, sig1)
nu_distrib = MultivariateNormal(m2, sig2)

n = 10000 # number of samples
mu_samples = mu_distrib.rsample(sample_shape=torch.Size([n]))
nu_samples = nu_distrib.rsample(sample_shape=torch.Size([n]))

# True Wasserstein
w = torch.norm(m1 - m2, p=2) + torch.trace(sig1 + sig2 - 2*torch.sqrt(torch.sqrt(sig1) * sig2 * torch.sqrt(sig1)))
print("true Wasserstein  :", w)

# Approximation of the Sliced Wasserstein
start = time.time()
sw_ap = sw_approx(mu_samples, nu_samples)
print(f"Approx SW : {sw_ap} ----- time : {time.time() - start} ---- approx error {torch.abs(sw_ap - w)}")

In [ ]:
mu_samples.size()

In [ ]:
def cost_xy(x, y, scaling_coef):
    '''
    L2 distance between vectors, using expanding and hence is more memory intensive
    :param x: x is tensor of shape [batch_size, time steps, features]
    :param y: y is tensor of shape [batch_size, time steps, features]
    :param scaling_coef: a scaling coefficient for distance between x and y
    :return: cost matrix: a matrix of size [batch_size, batch_size] where
    '''
    x = torch.unsqueeze(x, 1)
    y = torch.unsqueeze(y, 0)
    sum_over_pixs = torch.sum((x - y)**2, -1) * scaling_coef
    sum_over_time = torch.sum(sum_over_pixs, -1)
    return sum_over_time

def benchmark_sinkhorn(x, y, scaling_coef, epsilon=1.0, L=10, Lmin=10):
    '''
    Given two emprical measures with n points each with locations x and y
    outputs an approximation of the OT cost with regularization parameter epsilon
    niter is the max. number of steps in sinkhorn loop
    '''
    n_data = x.shape[0]

    # The Sinkhorn algorithm takes as input three variables :
    C = cost_xy(x, y, scaling_coef)  # Wasserstein cost function

    # both marginals are fixed with equal weights
    mu = 1.0 / torch.tensor(n_data, dtype=torch.float32) * torch.ones(n_data, dtype=torch.float32)
    nu = 1.0 / torch.tensor(n_data, dtype=torch.float32) * torch.ones(n_data, dtype=torch.float32)

    # Parameters of the Sinkhorn algorithm.
    thresh = 10**(-2)  # stopping criterion

    # Elementary operations .....................................................................
    def M(u, v):
        '''
        Modified cost for logarithmic updates
        $M_{ij} = (-c_{ij} + u_i + v_j) / \epsilon$
        '''
        return (-C + u[:,None] + v[None,:]) / epsilon

    def lse(A):
        '''
        log-sum-exp
        '''
        return A.logsumexp(dim=1, keepdim=True)
        # return tf.math.log(tf.reduce_sum(tf.exp(A), axis=1, keepdims=True) + 1e-6)  # add 10^-6 to prevent NaN

    # Actual Sinkhorn loop ......................................................................
    u, v, err = 0. * mu, 0. * nu, 0.

    for i in range(L):
        u1 = u  # useful to check the update
        u = epsilon * (torch.log(mu) - torch.squeeze(lse(M(u, v)))) + u
        #print(M(u, v).transpose)
        v = epsilon * (torch.log(nu) - torch.squeeze(lse(torch.transpose(M(u, v), 0, 1)))) + v
        err =torch.sum(torch.abs(u - u1))
        if torch.greater(torch.tensor(thresh), err) and i >= Lmin:
            break
    U, V = u, v
    pi = torch.exp(M(U, V))  # Transport plan pi = diag(a)*K*diag(b)
    cost = torch.sum(pi * C)  # Sinkhorn cost
    return cost

In [ ]:
benchmark_sinkhorn(x, y, 0.05)